<a href="https://colab.research.google.com/github/beaumontslim/misc/blob/main/pytorch_nlp_rest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
from flask import Flask, request
import pickle
import numpy as np
import torch
import torch.nn as nn
from torch.nn import functional as F
import pandas as pd
import numpy as np


In [17]:
input_size = 467
output_size= 2
hidden_size = 500


In [6]:
!ls

sample_data  text_class_pytorch  tfidfmodel.pickle


In [18]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.fc1 = torch.nn.Linear(input_size, hidden_size)
    self.fc2 = torch.nn.Linear(hidden_size, hidden_size)
    self.fc3 = torch.nn.Linear(hidden_size, output_size)

  def forward (self, X):
    X = torch.relu((self.fc1(X)))
    X = torch.relu((self.fc2(X)))
    X = self.fc3(X)

    return F.log_softmax(X,dim=1)

model = Net()

In [19]:
model.load_state_dict(torch.load('text_class_pytorch'))

<All keys matched successfully>

In [9]:
!pip install flask-ngrok

In [20]:
from flask_ngrok import run_with_ngrok

app = Flask(__name__)

run_with_ngrok(app)

In [21]:
with open('tfidfmodel.pickle','rb') as file:
  tfidf = pickle.load(file)

In [22]:
@app.route('/predict',methods=['POST'])
def customer_behavior():
    request_data = request.get_json(force=True)
    text = request_data['sentence']
    print("printing the sentence")
    print(text)
    text_list=[]
    text_list.append(text)
    print(text_list)
    numeric_text = tfidf.transform(text_list).toarray()
    output = model(torch.from_numpy(numeric_text).float())
    print("Printing predictions")
    print(output[:,0][0])
    print(output[:,1][0])
    sentiment="unknown"
    if torch.gt(output[:,0][0],output[:,1][0]):
      print("negative prediction")
      sentiment="negative"
    else:
      print("positive")
      sentiment="positive prediction"
    print("Printing prediction")     
    print(sentiment)
    return "The prediction is {}".format(sentiment)

In [23]:
app.run()

#note, you must add /predict to your post in post, and you must enter in raw code to get output for your prediction "ie" 
#{
#    "sentence": "good batting by America"
#}

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://a2749b73758e.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [24/Mar/2021 00:36:37] "POST / HTTP/1.1" 404 -
127.0.0.1 - - [24/Mar/2021 00:37:18] "POST /predict HTTP/1.1" 400 -
127.0.0.1 - - [24/Mar/2021 00:39:26] "POST /predict HTTP/1.1" 200 -


printing the sentence
good batting by America
['good batting by America']
Printing predictions
tensor(-1.6025, grad_fn=<SelectBackward>)
tensor(-0.2249, grad_fn=<SelectBackward>)
positive
Printing prediction
positive prediction
